## Make coco splits

In [2]:
from pylabel import importer, exporter, dataset
import numpy as np
from sklearn.model_selection import train_test_split
import json
import os
import pandas as pd

#### Save the entire dataset to coco if needed

In [3]:
voc_path = '/home/gridsan/manderson/ovdsat/data/simd'

In [17]:
mydataset = importer.ImportVOC(path = voc_path+'/Annotations', path_to_images=voc_path+'/JPEGImages', name='SIMD')


Importing VOC files...: 100%|██████████| 5000/5000 [00:05<00:00, 864.75it/s] 


In [18]:
mydataset.df.head()

,img_folder,img_filename,img_path,img_id,img_width,img_height,img_depth,ann_segmented,ann_bbox_xmin,ann_bbox_ymin,...,ann_iscrowd,ann_keypoints,ann_pose,ann_truncated,ann_difficult,cat_id,cat_name,cat_supercategory,split,annotated
id,,,,,,,,,,,,,,,,,,,,,
0,/home/gridsan/manderson/ovdsat/data/simd/JPEGI...,4657,NaN,0,1024,768,3,0,794.0,40.0,...,NaN,NaN,Unspecified,,,0,car,NaN,,1
1,/home/gridsan/manderson/ovdsat/data/simd/JPEGI...,4657,NaN,0,1024,768,3,0,702.0,57.0,...,NaN,NaN,Unspecified,,,0,car,NaN,,1
2,/home/gridsan/manderson/ovdsat/data/simd/JPEGI...,4657,NaN,0,1024,768,3,0,620.0,91.0,...,NaN,NaN,Unspecified,,,0,car,NaN,,1
3,/home/gridsan/manderson/ovdsat/data/simd/JPEGI...,4657,NaN,0,1024,768,3,0,522.0,110.0,...,NaN,NaN,Unspecified,,,0,car,NaN,,1
4,/home/gridsan/manderson/ovdsat/data/simd/JPEGI...,4657,NaN,0,1024,768,3,0,586.0,173.0,...,NaN,NaN,Unspecified,,,1,truck,NaN,,1


In [19]:
# FOR SIMD ONLY: fix categories!
print(len(mydataset.df))
mydataset.df.loc[mydataset.df['cat_id'] == '14', 'cat_id'] = '11'
mydataset.df.loc[mydataset.df['cat_id'] == '16', 'cat_id'] = '10'
mydataset.df['img_filename'] = mydataset.df['img_filename'] + '.jpg'
print(len(mydataset.df))

45310
45310


In [20]:
mydataset.df.head()

,img_folder,img_filename,img_path,img_id,img_width,img_height,img_depth,ann_segmented,ann_bbox_xmin,ann_bbox_ymin,...,ann_iscrowd,ann_keypoints,ann_pose,ann_truncated,ann_difficult,cat_id,cat_name,cat_supercategory,split,annotated
id,,,,,,,,,,,,,,,,,,,,,
0,/home/gridsan/manderson/ovdsat/data/simd/JPEGI...,4657.jpg,NaN,0,1024,768,3,0,794.0,40.0,...,NaN,NaN,Unspecified,,,0,car,NaN,,1
1,/home/gridsan/manderson/ovdsat/data/simd/JPEGI...,4657.jpg,NaN,0,1024,768,3,0,702.0,57.0,...,NaN,NaN,Unspecified,,,0,car,NaN,,1
2,/home/gridsan/manderson/ovdsat/data/simd/JPEGI...,4657.jpg,NaN,0,1024,768,3,0,620.0,91.0,...,NaN,NaN,Unspecified,,,0,car,NaN,,1
3,/home/gridsan/manderson/ovdsat/data/simd/JPEGI...,4657.jpg,NaN,0,1024,768,3,0,522.0,110.0,...,NaN,NaN,Unspecified,,,0,car,NaN,,1
4,/home/gridsan/manderson/ovdsat/data/simd/JPEGI...,4657.jpg,NaN,0,1024,768,3,0,586.0,173.0,...,NaN,NaN,Unspecified,,,1,truck,NaN,,1


In [21]:
exp = exporter.Export(mydataset)
exp.ExportToCoco(output_path=f'{voc_path}/simd_coco_all.json', cat_id_index=0)

/home/gridsan/manderson/.conda/envs/test/lib/python3.9/site-packages/pylabel/exporter.py:767: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r"^\s*$", np.nan, regex=True)
Exporting to COCO file...: 100%|██████████| 45310/45310 [00:32<00:00, 1392.52it/s]


['/home/gridsan/manderson/ovdsat/data/simd/simd_coco_all.json']

In [10]:
# Fix .xml -> jpg, if needed

coco_json = "/home/gridsan/manderson/ovdsat/data/mar/mar_coco_all.json"
images_dir = "/home/gridsan/manderson/ovdsat/data/mar/JPEGImages"
out_json = "/home/gridsan/manderson/ovdsat/data/mar/mar_coco_all_fixed.json"

with open(coco_json, "r") as f:
    coco = json.load(f)

num_fixed = 0
num_missing = 0

for img in coco["images"]:
    fname = img["file_name"]

    if fname.lower().endswith(".xml"):
        new_fname = fname[:-4] + ".jpg"
        jpg_path = os.path.join(images_dir, new_fname)

        if os.path.exists(jpg_path):
            img["file_name"] = new_fname
            num_fixed += 1
        else:
            print(f"[WARN] JPG not found for {fname}")
            num_missing += 1

with open(out_json, "w") as f:
    json.dump(coco, f)

print(f"Fixed {num_fixed} image entries")
print(f"Missing JPGs: {num_missing}")
print(f"Saved to: {out_json}")

Fixed 589 image entries
Missing JPGs: 0
Saved to: /home/gridsan/manderson/ovdsat/data/mar/mar_coco_all_fixed.json


In [44]:
for M in [1, 2, 3]:
    in_path = f'simd/old/train_coco_finetune_val-{M}.json'
    out_path = f'simd/train_coco_finetune_val-{M}.json'
    with open(in_path, "r") as f:
        coco = json.load(f)

    for img in coco["images"]:
        fname = img["file_name"]
        if not fname.lower().endswith(".jpg"):
            img["file_name"] = fname + ".jpg"

    with open(out_path, "w") as f:
        json.dump(coco, f, indent=4)

    print(f"Saved updated COCO to {out_path}")

Saved updated COCO to simd/train_coco_finetune_val-1.json
Saved updated COCO to simd/train_coco_finetune_val-2.json
Saved updated COCO to simd/train_coco_finetune_val-3.json


### Generate splits for few shot, finetune, and validation

In [22]:
# Load your COCO dataset
coco_json = f"/home/gridsan/manderson/ovdsat/data/simd/simd_coco_all.json"
path_to_images = "/home/gridsan/manderson/ovdsat/data/simd/JPEGImages"   # folder with your images

all_dataset = importer.ImportCoco(
    path=coco_json,
    path_to_images=path_to_images,
    name="SIMD"
)

# Get the dataframe
df = all_dataset.df
print(len(df))
df.head()

45310


,img_folder,img_filename,img_path,img_id,img_width,img_height,img_depth,ann_segmented,ann_bbox_xmin,ann_bbox_ymin,...,ann_iscrowd,ann_keypoints,ann_pose,ann_truncated,ann_difficult,cat_id,cat_name,cat_supercategory,split,annotated
id,,,,,,,,,,,,,,,,,,,,,
0,/home/gridsan/manderson/ovdsat/data/simd/JPEGI...,4657.jpg,,0,1024,768,3,0,794.0,40.0,...,0.0,,Unspecified,,,0,car,,,1
1,/home/gridsan/manderson/ovdsat/data/simd/JPEGI...,4657.jpg,,0,1024,768,3,0,702.0,57.0,...,0.0,,Unspecified,,,0,car,,,1
2,/home/gridsan/manderson/ovdsat/data/simd/JPEGI...,4657.jpg,,0,1024,768,3,0,620.0,91.0,...,0.0,,Unspecified,,,0,car,,,1
3,/home/gridsan/manderson/ovdsat/data/simd/JPEGI...,4657.jpg,,0,1024,768,3,0,522.0,110.0,...,0.0,,Unspecified,,,0,car,,,1
4,/home/gridsan/manderson/ovdsat/data/simd/JPEGI...,4657.jpg,,0,1024,768,3,0,586.0,173.0,...,0.0,,Unspecified,,,1,truck,,,1


In [23]:
save_dir = '/home/gridsan/manderson/ovdsat/data/simd'

for M in [1, 2, 3]:
    img_ids = set()
    prev_samples = {}  # track selected samples per category
    
    for N in [1, 3, 5, 10, 30]:
        train_file = f'{save_dir}/train_coco_subset_N{N}-{M}.json'
        val_file = f'{save_dir}/val_coco-{M}.json'
        finetune_val_file = f'{save_dir}/train_coco_finetune_val-{M}.json'
        
        N_samples_list = []

        for value, group in df.groupby('cat_id'):
            #print(f"cat_id = {value}")
            # start with previous samples for this category, if any
            prev_df = prev_samples.get(value, pd.DataFrame(columns=group.columns))
            pool = group.loc[~group['img_id'].isin(prev_df['img_id'])]

            # number of new samples to add for this N
            needed = max(0, N - len(prev_df))
            pool = group.loc[~group['img_id'].isin(prev_df['img_id'])]
            take = min(needed, len(pool))
            if take < needed:
                print(f'Warning: for cat id {value}, N={N}, M={M}, there are only {take} images left to sample from!')
            
            # print(
            #     f"cat_id={value} | "
            #     f"group={len(group)} | "
            #     f"prev={len(prev_df)} | "
            #     f"needed={needed} | "
            #     f"remaining img pool={len(pool)}"
            # )
            
            if take > 0:
                # new_samples = group.loc[~group['img_id'].isin(prev_df['img_id'])].sample(
                #     n=min(needed, len(group) - len(prev_df)), replace=False
                # )
                # current_df = pd.concat([prev_df, new_samples], ignore_index=True)
                new_samples = pool.sample(n=take, replace=False)
                current_df = pd.concat([prev_df, new_samples], ignore_index=True)
            else:
                current_df = prev_df

            # save for next round
            prev_samples[value] = current_df
            N_samples_list.append(current_df)
            img_ids.update(current_df['img_id'].tolist())

        print(f"M={M}, N={N}, total unique imgs={len(img_ids)}")

        # combine and export
        N_samples_df = pd.concat(N_samples_list, ignore_index=True).reset_index(drop=True)
        pylabel_df_N = dataset.Dataset(N_samples_df)
        exp = exporter.Export(pylabel_df_N)
        exp.ExportToCoco(output_path=train_file, cat_id_index=0)

        with open(train_file, 'r') as f:
            train_data = json.load(f)
        train_data['categories'] = sorted(train_data['categories'], key=lambda x: x['id'])
        with open(f'{train_file[:-5]}.json', 'w') as f:
            json.dump(train_data, f, indent=4)
        print(f'Saved {train_file[:-5]}.json')

    # Remaining images (not in any N-shot)
    remaining_df = df[~df['img_id'].isin(img_ids)]
    remaining_img_ids = remaining_df['img_id'].unique()

    # Split into validation and finetune sets
    ft_percent = 0.1
    val_ids, ft_ids = train_test_split(remaining_img_ids, test_size=ft_percent)

    # --- Validation set ---
    val_df = df[df['img_id'].isin(val_ids)].reset_index(drop=True)
    exp = exporter.Export(dataset.Dataset(val_df))
    exp.ExportToCoco(output_path=val_file, cat_id_index=0)

    # --- Finetune validation set ---
    ft_df = df[df['img_id'].isin(ft_ids)].reset_index(drop=True)
    exp = exporter.Export(dataset.Dataset(ft_df))
    exp.ExportToCoco(output_path=finetune_val_file, cat_id_index=0)

    # Sort categories in both
    for fpath in [finetune_val_file, val_file]:
        with open(fpath, 'r') as f:
            data = json.load(f)
        data['categories'] = sorted(data['categories'], key=lambda x: x['id'])
        with open(f'{fpath[:-5]}.json', 'w') as f:
            json.dump(data, f, indent=4)
        print(f'Saved {fpath[:-5]}.json')


/state/partition1/slurm_tmp/4037687.0.0/ipykernel_117667/3755223793.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  current_df = pd.concat([prev_df, new_samples], ignore_index=True)
/state/partition1/slurm_tmp/4037687.0.0/ipykernel_117667/3755223793.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  current_df = pd.concat([prev_df, new_samples], ignore_index=True)
/state/partition1/slurm_tmp/4037687.0.0/ipykernel_117667/3755223793.py:41: FutureWarning: The behavior of DataFrame concatenat

M=1, N=1, total unique imgs=15


Exporting to COCO file...: 100%|██████████| 15/15 [00:00<00:00, 995.67it/s]
/home/gridsan/manderson/.conda/envs/test/lib/python3.9/site-packages/pylabel/exporter.py:767: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r"^\s*$", np.nan, regex=True)


Saved /home/gridsan/manderson/ovdsat/data/simd/train_coco_subset_N1-1.json
M=1, N=3, total unique imgs=45


Exporting to COCO file...: 100%|██████████| 45/45 [00:00<00:00, 1851.77it/s]
/home/gridsan/manderson/.conda/envs/test/lib/python3.9/site-packages/pylabel/exporter.py:767: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r"^\s*$", np.nan, regex=True)


...

M=3, N=30, total unique imgs=421


Exporting to COCO file...: 100%|██████████| 450/450 [00:00<00:00, 1936.06it/s]


Saved /home/gridsan/manderson/ovdsat/data/simd/train_coco_subset_N30-3.json


Exporting to COCO file...: 100%|██████████| 3840/3840 [00:01<00:00, 3602.80it/s]


Saved /home/gridsan/manderson/ovdsat/data/simd/train_coco_finetune_val-3.json
Saved /home/gridsan/manderson/ovdsat/data/simd/val_coco-3.json


### Extras

In [5]:
### Sanity check for N=100

save_dir = '/home/gridsan/manderson/ovdsat/data/dior'

for M in [1, 2, 3]:
    img_ids = set()
    prev_samples = {}  # track selected samples per category
    
    for N in [100]:
        train_file = f'{save_dir}/train_coco_subset_N{N}-{M}.json'
        val_file = f'{save_dir}/val_coco_N{N}-{M}.json'
        finetune_val_file = f'{save_dir}/train_coco_finetune_val_N{N}-{M}.json'
        
        N_samples_list = []

        for value, group in df.groupby('cat_id'):
            # start with previous samples for this category, if any
            prev_df = prev_samples.get(value, pd.DataFrame(columns=group.columns))

            # number of new samples to add for this N
            needed = max(0, N - len(prev_df))
            if needed > 0:
                new_samples = group.loc[~group['img_id'].isin(prev_df['img_id'])].sample(
                    n=min(needed, len(group) - len(prev_df)), replace=False
                )
                current_df = pd.concat([prev_df, new_samples], ignore_index=True)
            else:
                current_df = prev_df

            # save for next round
            prev_samples[value] = current_df
            N_samples_list.append(current_df)
            img_ids.update(current_df['img_id'].tolist())

        print(f"M={M}, N={N}, total unique imgs={len(img_ids)}")

        # combine and export
        N_samples_df = pd.concat(N_samples_list, ignore_index=True).reset_index(drop=True)
        pylabel_df_N = dataset.Dataset(N_samples_df)
        exp = exporter.Export(pylabel_df_N)
        exp.ExportToCoco(output_path=train_file, cat_id_index=0)

        with open(train_file, 'r') as f:
            train_data = json.load(f)
        train_data['categories'] = sorted(train_data['categories'], key=lambda x: x['id'])
        with open(f'{train_file[:-5]}.json', 'w') as f:
            json.dump(train_data, f, indent=4)
        print(f'Saved {train_file[:-5]}.json')

    # Remaining images (not in any N-shot)
    remaining_df = df[~df['img_id'].isin(img_ids)]
    remaining_img_ids = remaining_df['img_id'].unique()

    # Split into validation and finetune sets
    ft_percent = 0.1
    val_ids, ft_ids = train_test_split(remaining_img_ids, test_size=ft_percent)

    # --- Validation set ---
    val_df = df[df['img_id'].isin(val_ids)].reset_index(drop=True)
    exp = exporter.Export(dataset.Dataset(val_df))
    exp.ExportToCoco(output_path=val_file, cat_id_index=0)

    # --- Finetune validation set ---
    ft_df = df[df['img_id'].isin(ft_ids)].reset_index(drop=True)
    exp = exporter.Export(dataset.Dataset(ft_df))
    exp.ExportToCoco(output_path=finetune_val_file, cat_id_index=0)

    # Sort categories in both
    for fpath in [finetune_val_file, val_file]:
        with open(fpath, 'r') as f:
            data = json.load(f)
        data['categories'] = sorted(data['categories'], key=lambda x: x['id'])
        with open(f'{fpath[:-5]}.json', 'w') as f:
            json.dump(data, f, indent=4)
        print(f'Saved {fpath[:-5]}.json')


/state/partition1/slurm_tmp/3935540.0.0/ipykernel_2065714/2750094772.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  current_df = pd.concat([prev_df, new_samples], ignore_index=True)
/state/partition1/slurm_tmp/3935540.0.0/ipykernel_2065714/2750094772.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  current_df = pd.concat([prev_df, new_samples], ignore_index=True)
/state/partition1/slurm_tmp/3935540.0.0/ipykernel_2065714/2750094772.py:26: FutureWarning: The behavior of DataFrame concate

M=1, N=100, total unique imgs=1916


Exporting to COCO file...: 100%|██████████| 2000/2000 [00:01<00:00, 1029.06it/s]


Saved /home/gridsan/manderson/ovdsat/data/dior/train_coco_subset_N100-1.json


Exporting to COCO file...: 100%|██████████| 3238/3238 [00:01<00:00, 3194.98it/s]


...

Saved /home/gridsan/manderson/ovdsat/data/dior/val_coco_N100-2.json
M=3, N=100, total unique imgs=1909


Exporting to COCO file...: 100%|██████████| 2000/2000 [00:00<00:00, 2517.00it/s]


Saved /home/gridsan/manderson/ovdsat/data/dior/train_coco_subset_N100-3.json


Exporting to COCO file...: 100%|██████████| 3244/3244 [00:00<00:00, 3300.66it/s]


Saved /home/gridsan/manderson/ovdsat/data/dior/train_coco_finetune_val_N100-3.json
Saved /home/gridsan/manderson/ovdsat/data/dior/val_coco_N100-3.json


In [3]:
import json
import random
from pathlib import Path

save_dir = Path("/home/gridsan/manderson/ovdsat/data/dior")

def extract_prefix(filename):
    return filename.split('_')[0]

def make_N20_from_N10_and_N30(M, save_dir=save_dir, seed=0):
    random.seed(seed)

    path_10 = save_dir / f"train_coco_subset_N10-{M}.json"
    path_30 = save_dir / f"train_coco_subset_N30-{M}.json"
    out_path = save_dir / f"train_coco_subset_N20-{M}.json"

    print(f"\n=== M={M}: building N=20 from {path_10.name} and {path_30.name} ===")

    with open(path_10, "r") as f:
        coco10 = json.load(f)
    with open(path_30, "r") as f:
        coco30 = json.load(f)

    # Map image_id → image metadata
    images_by_id = {img["id"]: img for img in coco30["images"]}
    for img in coco10["images"]:
        images_by_id.setdefault(img["id"], img)

    # Group annotations by category
    anns10_by_cat = {}
    anns30_by_cat = {}
    for ann in coco10["annotations"]:
        anns10_by_cat.setdefault(ann["category_id"], []).append(ann)
    for ann in coco30["annotations"]:
        anns30_by_cat.setdefault(ann["category_id"], []).append(ann)

    categories = sorted(coco30["categories"], key=lambda x: x["id"])
    final_annotations = []

    for cat in categories:
        cid = cat["id"]

        anns10 = anns10_by_cat.get(cid, [])
        anns30 = anns30_by_cat.get(cid, [])

        if len(anns10) != 10:
            raise ValueError(f"M={M}, category {cid}: N10 has {len(anns10)} anns (expected 10)")
        if len(anns30) < 30:
            raise ValueError(f"M={M}, category {cid}: N30 has {len(anns30)} anns (expected ≥30)")

        # Get prefixes used in N10
        used_prefixes = {
            extract_prefix(images_by_id[ann["image_id"]]["file_name"])
            for ann in anns10
        }

        # Filter N30 to exclude any with matching prefix
        eligible_from_30 = [
            ann for ann in anns30
            if extract_prefix(images_by_id[ann["image_id"]]["file_name"]) not in used_prefixes
        ]

        if len(eligible_from_30) < 10:
            raise ValueError(
                f"M={M}, category {cid}: only {len(eligible_from_30)} eligible anns after filtering by prefix"
            )

        extra = random.sample(eligible_from_30, 10)
        chosen = anns10 + extra
        final_annotations.extend(chosen)

    # Assign new unique annotation IDs
    new_annotations = []
    next_ann_id = 1
    for ann in final_annotations:
        ann_copy = dict(ann)
        ann_copy["id"] = next_ann_id
        new_annotations.append(ann_copy)
        next_ann_id += 1

    # Collect used images
    used_img_ids = {ann["image_id"] for ann in new_annotations}
    final_images = [images_by_id[iid] for iid in used_img_ids]

    coco20 = {
        "images": final_images,
        "annotations": new_annotations,
        "categories": categories,
    }

    for key in ("info", "licenses"):
        if key in coco30:
            coco20[key] = coco30[key]

    with open(out_path, "w") as f:
        json.dump(coco20, f, indent=4)

    print(f"[✓] wrote {out_path} — total annotations: {len(new_annotations)}")


# Run for all M
for M in [1, 2, 3, 4, 5]:
    make_N20_from_N10_and_N30(M)


=== M=1: building N=20 from train_coco_subset_N10-1.json and train_coco_subset_N30-1.json ===
[✓] wrote /home/gridsan/manderson/ovdsat/data/dior/train_coco_subset_N20-1.json — total annotations: 400

=== M=2: building N=20 from train_coco_subset_N10-2.json and train_coco_subset_N30-2.json ===
[✓] wrote /home/gridsan/manderson/ovdsat/data/dior/train_coco_subset_N20-2.json — total annotations: 400

=== M=3: building N=20 from train_coco_subset_N10-3.json and train_coco_subset_N30-3.json ===
[✓] wrote /home/gridsan/manderson/ovdsat/data/dior/train_coco_subset_N20-3.json — total annotations: 400

=== M=4: building N=20 from train_coco_subset_N10-4.json and train_coco_subset_N30-4.json ===
[✓] wrote /home/gridsan/manderson/ovdsat/data/dior/train_coco_subset_N20-4.json — total annotations: 400

=== M=5: building N=20 from train_coco_subset_N10-5.json and train_coco_subset_N30-5.json ===
[✓] wrote /home/gridsan/manderson/ovdsat/data/dior/train_coco_subset_N20-5.json — total annotations: 400


### Fix MAR20 names

In [1]:
import os
import json

In [3]:
CATEGORY_MAP = {
    "A1": "SU-35",
    "A2": "C-130",
    "A3": "C-17",
    "A4": "C-5",
    "A5": "F-16",
    "A6": "TU-160",
    "A7": "E-3",
    "A8": "B-52",
    "A9": "P-3C",
    "A10": "B-1B",
    "A11": "E-8",
    "A12": "TU-22",
    "A13": "F-15",
    "A14": "KC-135",
    "A15": "F-22",
    "A16": "FA-18",
    "A17": "TU-95",
    "A18": "KC-1",
    "A19": "SU-34",
    "A20": "SU-24",
}

def fix_category_names(json_dir):
    for fname in os.listdir(json_dir):
        if not fname.endswith(".json"):
            continue
        print(fname)

        path = os.path.join(json_dir, fname)

        with open(path, "r") as f:
            data = json.load(f)

        if "categories" not in data:
            print(f"[WARN] No categories in {fname}")
            continue

        changed = False
        for cat in data["categories"]:
            old_name = cat.get("name")
            if old_name in CATEGORY_MAP:
                cat["name"] = CATEGORY_MAP[old_name]
                changed = True

        if changed:
            with open(path, "w") as f:
                json.dump(data, f, indent=2)
            print(f"[OK] Updated {fname}")
        else:
            print(f"[SKIP] No changes in {fname}")

In [5]:
fix_category_names('mar')

[OK] Updated train_coco_subset_N3-3.json
[OK] Updated val_coco-3.json
[OK] Updated train_coco_finetune_val-1.json
[OK] Updated mar_coco_all.json
[OK] Updated train_coco_subset_N3-1.json
[OK] Updated train_coco_subset_N30-2.json
[OK] Updated train_coco_subset_N1-2.json
[OK] Updated train_coco_subset_N1-1.json
[OK] Updated val_coco-2.json
[OK] Updated train_coco_subset_N5-3.json
[OK] Updated train_coco_subset_N1-3.json
[OK] Updated train_coco_subset_N10-1.json
[OK] Updated val_coco-1.json
[OK] Updated train_coco_subset_N5-1.json
[OK] Updated train_coco_finetune_val-2.json
[OK] Updated train_coco_subset_N30-1.json
[OK] Updated train_coco_finetune_val-3.json
[OK] Updated train_coco_subset_N5-2.json
[OK] Updated train_coco_subset_N3-2.json
[OK] Updated train_coco_subset_N30-3.json
[OK] Updated train_coco_subset_N10-2.json
[OK] Updated train_coco_subset_N10-3.json


In [14]:
# Fix crop folder names
def rename_class_folders(root_dir, dry_run=False):
    """
    Recursively rename folders A1..A20 → aircraft names.

    Args:
        root_dir (str): Root directory to process
        dry_run (bool): If True, only print what would change
    """
    for dirpath, dirnames, _ in os.walk(root_dir, topdown=False):
        for d in dirnames:
            if d in CATEGORY_MAP:
                old_path = os.path.join(dirpath, d)
                new_path = os.path.join(dirpath, CATEGORY_MAP[d])

                if os.path.exists(new_path):
                    print(f"[SKIP] {new_path} already exists")
                    continue

                if dry_run:
                    print(f"[DRY-RUN] {old_path} → {new_path}")
                else:
                    os.rename(old_path, new_path)
                    print(f"[OK] {old_path} → {new_path}")

In [16]:
rename_class_folders('cropped_data/mar', dry_run=False)

[OK] cropped_data/mar/train/mar_N1-1/A2 → cropped_data/mar/train/mar_N1-1/C-130
[OK] cropped_data/mar/train/mar_N1-1/A14 → cropped_data/mar/train/mar_N1-1/KC-135
[OK] cropped_data/mar/train/mar_N1-1/A12 → cropped_data/mar/train/mar_N1-1/TU-22
[OK] cropped_data/mar/train/mar_N1-1/A9 → cropped_data/mar/train/mar_N1-1/P-3C
[OK] cropped_data/mar/train/mar_N1-1/A17 → cropped_data/mar/train/mar_N1-1/TU-95
[OK] cropped_data/mar/train/mar_N1-1/A20 → cropped_data/mar/train/mar_N1-1/SU-24
[OK] cropped_data/mar/train/mar_N1-1/A15 → cropped_data/mar/train/mar_N1-1/F-22
[OK] cropped_data/mar/train/mar_N1-1/A7 → cropped_data/mar/train/mar_N1-1/E-3
[OK] cropped_data/mar/train/mar_N1-1/A11 → cropped_data/mar/train/mar_N1-1/E-8
[OK] cropped_data/mar/train/mar_N1-1/A1 → cropped_data/mar/train/mar_N1-1/SU-35
[OK] cropped_data/mar/train/mar_N1-1/A6 → cropped_data/mar/train/mar_N1-1/TU-160
[OK] cropped_data/mar/train/mar_N1-1/A13 → cropped_data/mar/train/mar_N1-1/F-15
[OK] cropped_data/mar/train/mar_N1-1/A